# Data sequence and schema diagrams


## Data Sequences
### File apis

```mermaid
---
title: Data sequence - POST /upload
---

sequenceDiagram
    Django->>S3: file key, content
    Django->>Core: file key
    Core->>Workers: file key
    Core->>Elastic: file key
    S3->>Workers: file content
    Workers->>Elastic: chunk key, content
```


### Chat APIs


```mermaid
---
title: Data sequence - POST /chat/vanilla
---

sequenceDiagram
    Django->>Core: ChatHistory.messages[]
    Core->>LLM API: ChatHistory.messages[]

```


```mermaid
---
title: Data sequence - POST /chat/rag
---

sequenceDiagram
    Django->> Core: ChatHistory.messages[], File[].uuid
    Core->>Embedding API: ChatHistory.messages[]
    Embedding API->>Core: ChatHistory.messages[].embeddings
    Elastic->>Core: File[].Chunk[].embeddings
    Core->>LLM API: ChatHistory.messages[].embeddings + File[].Chunk[].embeddings

```

## Schema

```mermaid
---
title: Django schema
---

erDiagram
    User }|--|{ UserGroup : "UserGroup.users"
    User {
        UUID uuid
        string name
    }
    UserGroup {
        UUID uuid
        string name
        UUID[] users
    }
    
    FileRecord }|--|| UserGroup: "FileRecord.owner"
    FileRecord {
        UUID uuid
        UUID owner
        string key 
    }

    UserGroup ||--|{ Message : "Mesage.owner"
    Message {
        UUID uuid
        UUID owner
        string text
        string role 
    }

    Message }|--|| ChatHistory: "ChatHistory.messages"
    UserGroup ||--|{ ChatHistory: "ChatHistory.owner"

    ChatHistory {
        UUID uuid
        string name
        UUID owner
        UUID[] messages
        UUID[] files_received
        UUID[] files_cited
    }

    ChatHistory ||--o{ FileRecord: "ChatHistory.files_received"
    ChatHistory ||--o{ FileRecord: "ChatHistory.files_cited"
```

```mermaid
---
title: Elastic schema
---

erDiagram
    File ||--o{ Chunk : "File.uuid"
    File {
        UUID uuid
    }
    Chunk {
        UUID uuid
        UUID parent_file_uuid
        int index
        str text
        dict metadata
        float[] embedding
    }
```